# MODELO SOLUCIÓN - XGBOOST OPTIMIZADO

Este notebook implementa un modelo XGBoost avanzado optimizado para aprovechar al máximo el hardware disponible:
- **Intel i7 12va generación**
- **NVIDIA RTX 4070**

## Características principales:
1. **XGBoost** - Gradient Boosting con optimización GPU
2. **Búsqueda exhaustiva** de hiperparámetros
3. **Análisis detallado** de rendimiento
4. **Importancia de características** automática
5. **Generación de submission** optimizada

## Ventajas de XGBoost:
- ✅ Mejor accuracy en datos tabulares
- ✅ Aceleración GPU nativa
- ✅ Resistente al overfitting
- ✅ Paralelización eficiente
- ✅ Importancia de características automática

In [2]:
# 0. INSTALACIÓN DE LIBRERÍAS NECESARIAS
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado correctamente")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ Error instalando {package}: {e}")
        return False

print("INSTALANDO LIBRERÍAS PARA MODELOS AVANZADOS")
print("="*50)

# Lista de paquetes requeridos
required_packages = [
    "xgboost",          # XGBoost para gradient boosting
    "lightgbm",         # LightGBM para gradient boosting
    "optuna",           # Para optimización de hiperparámetros (opcional)
]

installed_packages = []
failed_packages = []

for package in required_packages:
    print(f"Instalando {package}...")
    if install_package(package):
        installed_packages.append(package)
    else:
        failed_packages.append(package)

print(f"\n" + "="*50)
print("RESUMEN DE INSTALACIÓN")
print("="*50)
print(f"✅ Instalados exitosamente: {len(installed_packages)}")
for pkg in installed_packages:
    print(f"   - {pkg}")

if failed_packages:
    print(f"❌ Falló la instalación: {len(failed_packages)}")
    for pkg in failed_packages:
        print(f"   - {pkg}")
    print("\nPuedes instalar manualmente con:")
    for pkg in failed_packages:
        print(f"   pip install {pkg}")
else:
    print("\n🎉 ¡Todas las librerías instaladas correctamente!")
    print("✅ Listo para entrenar modelos avanzados")

print("="*50)

INSTALANDO LIBRERÍAS PARA MODELOS AVANZADOS
Instalando xgboost...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 6.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 MB 7.7 MB/s eta 0:00:0000:01m00:01
✅ xgboost instalado correctamente
Instalando lightgbm...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.5 MB/s eta 0:00:00a 0:00:01
✅ lightgbm instalado correctamente
Instalando optuna...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 23.8 MB/s eta 0:00:00
✅ optuna instalado correctamente

RESUMEN DE INSTALACIÓN
✅ Instalados exitosamente: 3
   - xgboost
   - lightgbm
   - optuna

🎉 ¡Todas las librerías

In [3]:
# 1. IMPORTAR LIBRERÍAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
import time
import warnings
import joblib
import os
import multiprocessing
from datetime import datetime
warnings.filterwarnings('ignore')

print("MODELOS AVANZADOS - XGBOOST Y LIGHTGBM")
print("="*60)
print("Aprovechando Intel i7 12va + RTX 4070")
print("="*60)

MODELOS AVANZADOS - XGBOOST Y LIGHTGBM
Aprovechando Intel i7 12va + RTX 4070


In [4]:
# 2. CARGAR Y PREPARAR DATOS
print("\nCargando datos...")
train = pd.read_csv('/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/Data/train_preprocessed.csv')
test = pd.read_csv('/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/Data/test_preprocessed.csv')

print(f"Train: {train.shape[0]:,} muestras, {train.shape[1]} características")
print(f"Test: {test.shape[0]:,} muestras, {test.shape[1]} características")

# Configurar variables
target_col = 'RENDIMIENTO_GLOBAL'
id_col = 'ID'

# Separar características y target
X = train.drop([id_col, target_col], axis=1)
y = train[target_col]
X_test = test.drop([id_col], axis=1, errors='ignore')

print(f"\nCaracterísticas para entrenamiento: {X.shape[1]}")
print(f"Distribución del target:")
print(y.value_counts().sort_index())

# Codificar target para modelos
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)
target_classes = le_target.classes_
n_classes = len(target_classes)

print(f"Clases codificadas: {dict(zip(range(n_classes), target_classes))}")


Cargando datos...
Train: 578,824 muestras, 21 características
Test: 296,786 muestras, 20 características

Características para entrenamiento: 19
Distribución del target:
RENDIMIENTO_GLOBAL
alto          132509
bajo          149170
medio-alto    146099
medio-bajo    151046
Name: count, dtype: int64
Clases codificadas: {0: 'alto', 1: 'bajo', 2: 'medio-alto', 3: 'medio-bajo'}


In [6]:
# 3. PREPARAR VARIABLES CATEGÓRICAS Y CONFIGURACIÓN
# Codificar variables categóricas si las hay
categorical_columns = X.select_dtypes(include=['object', 'bool']).columns
if len(categorical_columns) > 0:
    print(f"Codificando {len(categorical_columns)} variables categóricas...")
    le_dict = {}
    for col in categorical_columns:
        # Manejar valores faltantes
        X[col] = X[col].fillna('Missing')
        X_test[col] = X_test[col].fillna('Missing')
        
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        
        # Para test, manejar valores no vistos durante entrenamiento
        test_values = X_test[col].astype(str)
        test_encoded = []
        for val in test_values:
            if val in le.classes_:
                test_encoded.append(le.transform([val])[0])
            else:
                # Asignar a la clase 'Missing' si existe, sino a la primera clase
                if 'Missing' in le.classes_:
                    test_encoded.append(le.transform(['Missing'])[0])
                else:
                    test_encoded.append(0)  # Primera clase
        
        X_test[col] = test_encoded
        le_dict[col] = le

# Detectar recursos del sistema
n_cores = multiprocessing.cpu_count()
print(f"\n" + "="*50)
print("CONFIGURACIÓN DEL SISTEMA")
print("="*50)
print(f"CPU cores disponibles: {n_cores}")
print(f"Utilizando: {n_cores-1} cores para entrenamiento")

# Configuración de validación cruzada
cv_folds = 5
cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
print(f"Validación cruzada: {cv_folds} folds")
print("="*50)

Codificando 6 variables categóricas...

CONFIGURACIÓN DEL SISTEMA
CPU cores disponibles: 16
Utilizando: 15 cores para entrenamiento
Validación cruzada: 5 folds


In [7]:
# 4. MODELO XGBOOST CON OPTIMIZACIÓN GPU
print(f"\n" + "="*50)
print("MODELO 1: XGBOOST AVANZADO")
print("="*50)

# Parámetros base para XGBoost
xgb_params_base = {
    'objective': 'multi:softprob',
    'num_class': n_classes,
    'eval_metric': 'mlogloss',
    'random_state': 42,
    'n_jobs': n_cores-1,
    'tree_method': 'gpu_hist',  # Usar GPU si está disponible
    'gpu_id': 0,
    'verbosity': 1
}

# Espacio de hiperparámetros para XGBoost
xgb_param_dist = {
    'n_estimators': [200, 300, 500, 800],
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'subsample': [0.8, 0.9, 0.95, 1.0],
    'colsample_bytree': [0.8, 0.9, 0.95, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# Configuración de búsqueda
n_iter_xgb = 50  # Número de iteraciones
print(f"Espacio de búsqueda XGBoost: ~{np.prod([len(v) for v in xgb_param_dist.values()]):,} combinaciones")
print(f"Realizando {n_iter_xgb} iteraciones con {cv_folds} folds = {n_iter_xgb * cv_folds} entrenamientos")

xgb_model = xgb.XGBClassifier(**xgb_params_base)

xgb_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=xgb_param_dist,
    n_iter=n_iter_xgb,
    cv=cv,
    scoring='accuracy',
    n_jobs=2,  # Paralelización limitada por GPU
    verbose=2,
    random_state=42,
    return_train_score=False
)

print(f"Iniciando entrenamiento XGBoost...")
start_time_xgb = time.time()

try:
    xgb_search.fit(X, y_encoded)
    elapsed_xgb = time.time() - start_time_xgb
    
    print(f"XGBoost completado en {elapsed_xgb/60:.2f} minutos")
    print(f"Mejor score XGBoost: {xgb_search.best_score_:.4f}")
    print(f"Mejores parámetros XGBoost:")
    for param, value in xgb_search.best_params_.items():
        print(f"  {param}: {value}")
    
    best_xgb = xgb_search.best_estimator_
    xgb_available = True
    xgb_gpu_used = True
    
except Exception as e:
    print(f"Error con GPU XGBoost, usando CPU: {e}")
    # Fallback a CPU
    xgb_params_base.pop('tree_method', None)
    xgb_params_base.pop('gpu_id', None)
    
    xgb_model_cpu = xgb.XGBClassifier(**xgb_params_base)
    xgb_search_cpu = RandomizedSearchCV(
        xgb_model_cpu,
        param_distributions=xgb_param_dist,
        n_iter=30,  # Menos iteraciones en CPU
        cv=cv,
        scoring='accuracy',
        n_jobs=n_cores-1,
        verbose=1,
        random_state=42
    )
    
    xgb_search_cpu.fit(X, y_encoded)
    elapsed_xgb = time.time() - start_time_xgb
    best_xgb = xgb_search_cpu.best_estimator_
    xgb_search = xgb_search_cpu
    xgb_available = True
    xgb_gpu_used = False
    print(f"XGBoost CPU completado en {elapsed_xgb/60:.2f} minutos")


MODELO 1: XGBOOST AVANZADO
Espacio de búsqueda XGBoost: ~172,800 combinaciones
Realizando 50 iteraciones con 5 folds = 250 entrenamientos
Iniciando entrenamiento XGBoost...
Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:22:05] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  bst.update(dtrain, iteration=i, fobj=obj)
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:22:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:22:05] WARNING: /workspace/src/common/error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:

[CV] END colsample_bytree=0.95, gamma=0.3, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=800, reg_alpha=0.1, reg_lambda=1, subsample=0.95; total time=  29.5s
[CV] END colsample_bytree=0.95, gamma=0.3, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=800, reg_alpha=0.1, reg_lambda=1, subsample=0.95; total time=  29.5s


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:22:34] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:22:34] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [14:23:03] WARNING: /workspace/src/common/error_msg.cc:27: The

[CV] END colsample_bytree=0.95, gamma=0.3, learning_rate=0.05, max_depth=5, min_child_weight=1, n_estimators=800, reg_alpha=0.1, reg_lambda=1, subsample=0.95; total time=  29.2s


...

[CV] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=  10.9s


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:53:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [14:53:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


[CV] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=  10.8s


/home/jp/Repositorios/UDEA/InteligenciaArtificial-UdeA20251/.venv/lib/python3.12/site-packages/xgboost/core.py:2676: UserWarning: [14:53:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


[CV] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.05, max_depth=5, min_child_weight=5, n_estimators=300, reg_alpha=0, reg_lambda=1, subsample=0.8; total time=   5.5s
XGBoost completado en 31.48 minutos
Mejor score XGBoost: 0.4207
Mejores parámetros XGBoost:
  subsample: 1.0
  reg_lambda: 2
  reg_alpha: 0
  n_estimators: 800
  min_child_weight: 1
  max_depth: 6
  learning_rate: 0.1
  gamma: 0.1
  colsample_bytree: 0.9


In [10]:
# 6. RESULTADO DEL ENTRENAMIENTO XGBOOST
print(f"\n" + "="*60)
print("RESULTADO DEL ENTRENAMIENTO XGBOOST")
print("="*60)

# Definir variables necesarias para el resto del análisis
best_xgb_score = xgb_search.best_score_

# Información del modelo XGBoost
print(f"🎯 MODELO: XGBoost")
print(f"📊 Accuracy: {best_xgb_score:.4f} ({best_xgb_score*100:.2f}%)")
print(f"⏱️  Tiempo entrenamiento: {elapsed_xgb/60:.2f} minutos")
print(f"🔥 GPU utilizada: {'✅ SI' if xgb_gpu_used else '❌ NO'}")

print(f"\n🔧 Mejores hiperparámetros:")
for param, value in xgb_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\n🎯 SELECCIONADO PARA ANÁLISIS Y SUBMISSION: XGBoost")
print("="*60)


RESULTADO DEL ENTRENAMIENTO XGBOOST
🎯 MODELO: XGBoost
📊 Accuracy: 0.4207 (42.07%)
⏱️  Tiempo entrenamiento: 31.48 minutos
🔥 GPU utilizada: ✅ SI

🔧 Mejores hiperparámetros:
  subsample: 1.0
  reg_lambda: 2
  reg_alpha: 0
  n_estimators: 800
  min_child_weight: 1
  max_depth: 6
  learning_rate: 0.1
  gamma: 0.1
  colsample_bytree: 0.9

🎯 SELECCIONADO PARA ANÁLISIS Y SUBMISSION: XGBoost


In [17]:
# 9. GUARDAR RESULTADOS Y MODELO XGBOOST
print(f"\n" + "="*50)
print("GUARDANDO RESULTADOS - XGBOOST")
print("="*50)

# Verificar y corregir tipos de datos de X_test
print("Verificando tipos de datos para predicción...")
print("Columnas con tipo 'object' en X_test:")
object_columns = X_test.select_dtypes(include=['object']).columns
if len(object_columns) > 0:
    print(f"  {list(object_columns)}")
    
    # Convertir columnas object a category para XGBoost
    X_test_corrected = X_test.copy()
    for col in object_columns:
        if col in X_test_corrected.columns:
            X_test_corrected[col] = X_test_corrected[col].astype('category')
    print("  ✅ Tipos de datos corregidos")
else:
    X_test_corrected = X_test
    print("  ✅ Todos los tipos de datos son correctos")

# Realizar predicciones en test con tipos corregidos
print(f"Realizando predicciones finales...")
try:
    y_pred_test = best_xgb.predict(X_test_corrected)
    y_pred_proba_test = best_xgb.predict_proba(X_test_corrected)
    
    # Decodificar predicciones
    y_pred_test_decoded = le_target.inverse_transform(y_pred_test)
    
    # Análisis de confianza
    probas_max = np.max(y_pred_proba_test, axis=1)
    print(f"✅ Predicciones completadas exitosamente")
    print(f"  📊 Confianza promedio: {probas_max.mean():.3f}")
    print(f"  🎯 Predicciones alta confianza (>0.8): {(probas_max > 0.8).sum():,}")
    
except Exception as e:
    print(f"❌ Error en predicciones: {e}")
    # Crear submission básico solo con modelo guardado
    print("Guardando solo el modelo entrenado...")

# Crear directorio con timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"Resultados_Modelos/XGBoost_Avanzado/XGBoost_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Guardar modelo XGBoost entrenado (LO MÁS IMPORTANTE)
model_path = os.path.join(output_dir, f"modelo_xgboost_avanzado.joblib")
joblib.dump(best_xgb, model_path)
print(f"✅ MODELO GUARDADO: {model_path}")

# Preparar IDs de test
if 'ID' in test.columns:
    test_ids = test['ID']
else:
    test_ids = range(len(X_test))

# Si las predicciones fueron exitosas, guardar submission
if 'y_pred_test_decoded' in locals():
    # Guardar submission principal
    submission = pd.DataFrame({
        'ID': test_ids,
        target_col: y_pred_test_decoded
    })
    
    submission_path = os.path.join(output_dir, f"submission_xgboost_avanzado.csv")
    submission.to_csv(submission_path, index=False)
    print(f"✅ SUBMISSION GUARDADO: {submission_path}")
    
    # Guardar submission con probabilidades
    prob_df = pd.DataFrame(y_pred_proba_test, columns=[f'prob_{classe}' for classe in target_classes])
    prob_df['ID'] = test_ids
    prob_df[target_col] = y_pred_test_decoded
    prob_df['confianza'] = probas_max
    
    prob_path = os.path.join(output_dir, f"submission_xgboost_probabilidades.csv")
    prob_df.to_csv(prob_path, index=False)
    print(f"✅ PROBABILIDADES GUARDADAS: {prob_path}")
else:
    print("⚠️  Sin predicciones - solo modelo guardado")

# Guardar importancia de características
if hasattr(best_xgb, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_xgb.feature_importances_
    }).sort_values('importance', ascending=False)
    
    importance_path = os.path.join(output_dir, "importancia_caracteristicas.csv")
    feature_importance.to_csv(importance_path, index=False)
    print(f"✅ IMPORTANCIA GUARDADA: {importance_path}")

# Guardar métricas y metadatos
metrics = {
    'modelo': 'XGBoost_Avanzado',
    'accuracy_cv': best_xgb_score,
    'tiempo_entrenamiento_min': elapsed_xgb / 60,
    'gpu_utilizada': xgb_gpu_used,
    'n_muestras_entrenamiento': len(X),
    'n_caracteristicas': X.shape[1],
    'n_clases': n_classes,
    'mejores_parametros': str(xgb_search.best_params_),
    'fecha_entrenamiento': timestamp
}

if 'probas_max' in locals():
    metrics.update({
        'confianza_promedio': probas_max.mean(),
        'confianza_alta_pct': (probas_max > 0.8).mean() * 100,
        'accuracy_cv_std': cv_scores.std(),
    })

metrics_df = pd.DataFrame(list(metrics.items()), columns=['Métrica', 'Valor'])
metrics_path = os.path.join(output_dir, f"metricas_xgboost_avanzado.csv")
metrics_df.to_csv(metrics_path, index=False)
print(f"✅ MÉTRICAS GUARDADAS: {metrics_path}")

print(f"\n🎉 ARCHIVOS GUARDADOS EN: {output_dir}")
print("="*50)
print("✅ PROCESO COMPLETADO - MODELO LISTO PARA USO")
print("="*50)


GUARDANDO RESULTADOS - XGBOOST
Verificando tipos de datos para predicción...
Columnas con tipo 'object' en X_test:
  ['FAMI_TIENEINTERNET']
  ✅ Tipos de datos corregidos
Realizando predicciones finales...
✅ Predicciones completadas exitosamente
  📊 Confianza promedio: 0.439
  🎯 Predicciones alta confianza (>0.8): 8,781
✅ MODELO GUARDADO: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640/modelo_xgboost_avanzado.joblib
✅ SUBMISSION GUARDADO: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640/submission_xgboost_avanzado.csv
✅ PROBABILIDADES GUARDADAS: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640/submission_xgboost_probabilidades.csv
✅ IMPORTANCIA GUARDADA: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640/importancia_caracteristicas.csv
✅ MÉTRICAS GUARDADAS: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640/metricas_xgboost_avanzado.csv

🎉 ARCHIVOS GUARDADOS EN: Resultados_Modelos/XGBoost_Avanzado/XGBoost_20250613_150640
✅ PROCESO COM